In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline
import xgboost as xgb

pal = sns.color_palette()

In [4]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [14]:
corpus = train.question1 + ". " + train.question2

vectorizer = TfidfVectorizer(min_df=1, ngram_range=(1, 2))
X_train = vectorizer.fit_transform(corpus.values.astype('U'))
y_train = train.is_duplicate

In [15]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_train, label=y_train)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 300, watchlist, early_stopping_rounds=50, verbose_eval=10)

[0]	train-logloss:0.690481	valid-logloss:0.690481
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.668636	valid-logloss:0.668636
[20]	train-logloss:0.652546	valid-logloss:0.652546
[30]	train-logloss:0.640207	valid-logloss:0.640207
[40]	train-logloss:0.630789	valid-logloss:0.630789
[50]	train-logloss:0.623236	valid-logloss:0.623236
[60]	train-logloss:0.61685	valid-logloss:0.61685
[70]	train-logloss:0.611535	valid-logloss:0.611535
[80]	train-logloss:0.606922	valid-logloss:0.606922
[90]	train-logloss:0.602919	valid-logloss:0.602919
[100]	train-logloss:0.59948	valid-logloss:0.59948
[110]	train-logloss:0.596221	valid-logloss:0.596221
[120]	train-logloss:0.593292	valid-logloss:0.593292
[130]	train-logloss:0.590627	valid-logloss:0.590627
[140]	train-logloss:0.58805	valid-logloss:0.58805
[150]	train-logloss:0.585681	valid-logloss:0.585681
[160]	train-logloss:0.583495	valid

In [1]:
corpus = test.question1 + ". " + test.question2

vectorizer = TfidfVectorizer(min_df=1, ngram_range=(1, 2))
X_test = vectorizer.fit_transform(corpus.values.astype('U'))

NameError: name 'test' is not defined

In [ ]:
d_test = xgb.DMatrix(X_test)
p_test = bst.predict(d_test)

sub = pd.DataFrame()
sub['test_id'] = test['test_id']
sub['is_duplicate'] = p_test
sub.to_csv('simple_xgb.csv', index=False)